In [1]:
import urllib.request, requests, re
ua_req = requests.get('https://www.whatismybrowser.com/detect/what-is-my-user-agent')
ua = re.search(r'analyse-my-user-agent=yes">(.+?)</a></div>', ua_req.text)
ua = ua.group(1)

# Code to retrieve NBA Teams

In [4]:
webpage = requests.get('https://en.wikipedia.org/wiki/National_Basketball_Association', headers = {'User-Agent': ua}).text
print(webpage)

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>National Basketball Association - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"559ad1da-49cf-4bdb-9234-80c7c138905a","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"National_Basketball_Association","wgTitle":"National Basketball Association","wgCurRevisionId":1058454004,"wgRevisionId":1058454004,"wgArticleId":22093,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Wikipedia pages semi-protected against vandalism","Wikipedia indefinitely move-protected pages","Arti

# Code to create a list of all 30 NBA teams

In [5]:
teams_raw = re.findall(r'<td><b><a href="/wiki/(.*?)</a></b>',webpage, re.DOTALL )

teams = list()

for line in teams_raw:
    m = re.search(r'>([\w]+.\w+.*)', line)
    m.group(0)
    teams.append(m.group(1))  

# Displaying all 30 NBA teams

In [6]:
teams

['Boston Celtics',
 'Brooklyn Nets',
 'New York Knicks',
 'Philadelphia 76ers',
 'Toronto Raptors',
 'Chicago Bulls',
 'Cleveland Cavaliers',
 'Detroit Pistons',
 'Indiana Pacers',
 'Milwaukee Bucks',
 'Atlanta Hawks',
 'Charlotte Hornets',
 'Miami Heat',
 'Orlando Magic',
 'Washington Wizards',
 'Denver Nuggets',
 'Minnesota Timberwolves',
 'Oklahoma City Thunder',
 'Portland Trail Blazers',
 'Utah Jazz',
 'Golden State Warriors',
 'Los Angeles Clippers',
 'Los Angeles Lakers',
 'Phoenix Suns',
 'Sacramento Kings',
 'Dallas Mavericks',
 'Houston Rockets',
 'Memphis Grizzlies',
 'New Orleans Pelicans',
 'San Antonio Spurs']

# Code that retrieves the 30 team's city and stadium

In [7]:
cities_stadiums = re.findall(r'<td><a href="/wiki/(.*?)</a>',webpage, re.DOTALL )

cities_stadiums_raw = list()

for line in cities_stadiums:
    n = re.search(r'>([\w]+.\w+.*)', line)
    n.group(0)
    cities_stadiums_raw.append(n.group(1))
    
cities = list()
stadiums = list()
counter = 1 

for item in cities_stadiums_raw:
    if counter%2 == 0:
        stadiums.append(item)
    else:
        cities.append(item)
    counter = counter + 1

# Displaying the Cities

In [8]:
print(cities)  

['Boston, Massachusetts', 'New York City, New York', 'New York City, New York', 'Philadelphia, Pennsylvania', 'Toronto, Ontario', 'Chicago, Illinois', 'Cleveland, Ohio', 'Detroit, Michigan', 'Indianapolis, Indiana', 'Milwaukee, Wisconsin', 'Atlanta, Georgia', 'Charlotte, North Carolina', 'Miami, Florida', 'Orlando, Florida', 'Washington, D.C.', 'Denver, Colorado', 'Minneapolis, Minnesota', 'Oklahoma City, Oklahoma', 'Portland, Oregon', 'Salt Lake City, Utah', 'San Francisco, California', 'Los Angeles, California', 'Los Angeles, California', 'Phoenix, Arizona', 'Sacramento, California', 'Dallas, Texas', 'Houston, Texas', 'Memphis, Tennessee', 'New Orleans, Louisiana', 'San Antonio, Texas']


# Displaying the Stadiums

In [9]:
print(stadiums)  

['TD Garden', 'Barclays Center', 'Madison Square Garden', 'Wells Fargo Center', 'Scotiabank Arena', 'United Center', 'Rocket Mortgage FieldHouse', 'Little Caesars Arena', 'Gainbridge Fieldhouse', 'Fiserv Forum', 'State Farm Arena', 'Spectrum Center', 'FTX Arena', 'Amway Center', 'Capital One Arena', 'Ball Arena', 'Target Center', 'Paycom Center', 'Moda Center', 'Vivint Arena', 'Chase Center', 'Staples Center', 'Staples Center', 'Footprint Center', 'Golden 1 Center', 'American Airlines Center', 'Toyota Center', 'FedExForum', 'Smoothie King Center', 'AT&amp;T Center']


# Main Code for Individual Player Data

In [10]:
playerList = []
for team in teams:
    team = team.replace(' ','_')
    teampage = 'https://en.wikipedia.org/wiki/' + team
    html = requests.get(teampage).text
    rex = '<h3><span class="mw-headline" id=[\"\w\_]+?oster">' + '.*?' + '</tbody></table>'
    print(team)
    chunk1 = re.search(rex, html, re.DOTALL).group(0)
    chunk1 = re.sub('\(basketball\)','',chunk1)
    chunk1 = re.sub('&amp;', '&', chunk1)
    rex = '<td style="text-align:center' + '(.+?)' + '</td></tr>'
    playerChunks = re.findall(rex, chunk1, re.DOTALL)
#     print(playerChunks)
    for i in playerChunks:
        team = team.replace('_',' ')
        players = re.findall(r'title="(.+?)">', i) #finds [postion, name, alma mater] for each player
        heights = re.search(r'(\d\.\d\d)', i)       #finds weight of each player
        weights = re.search(r'">(\d\d\d)&#', i)     #finds heights
        heights = float(heights.group(1))
        weights = int(weights.group(1))
        players.append(team)
        players.append(heights)
        players.append(weights)
    #     print(players)
        for player in players[:1][:]:
            name = players[1]
            name = name.replace('&#39;',"%27")
            print(name)
            name = name.split()
            link = '<a href="(/wiki.+?' + re.escape(name[0]) + r'.+?)"'
            link = re.search(link, i)
            url = 'https://en.wikipedia.org' + link.group(1)
            if url[-1] == '_':
                url = url + '(basketball)'
            html = requests.get(url).text
            try:
                nums = re.search(r'colspan="2"\S+?Career.+?</td>(.+?)</tr>', html, flags = re.DOTALL)
#                 print(nums.group(1))
                ppg = re.search(r'<td>([\d\.]+)\n</td>', nums.group(1), flags = re.DOTALL)
                ppg = float(ppg.group(1))
                stats = re.findall(r'<td.+?([\d\.\,]+?)<', nums.group(1))
        #         stats = [x for x in stats if x != '.']
                stats = [x.replace(',','') for x in stats]
                statistics = [float(x) for x in stats]
            except:
                stats = [0,0,0,0,0,0,0,0,0,0,]
                ppg = [0]
            players.extend(statistics)
            players.append(ppg)
        playerList.append(players) 
# playerList

Boston_Celtics
Jaylen Brown
Bruno Fernando
Sam Hauser
Juancho Hernangómez
Al Horford
Enes Freedom
Romeo Langford
Aaron Nesmith
Jabari Parker
Payton Pritchard
Josh Richardson
Dennis Schröder
Marcus Smart
Jayson Tatum
Brodric Thomas
Grant Williams 
Robert Williams 
Brooklyn_Nets
LaMarcus Aldridge
DeAndre%27 Bembry
Bruce Brown 
Jevon Carter
Nicolas Claxton
David Duke Jr.
Kevin Durant
Kessler Edwards
Blake Griffin
James Harden
Joe Harris 
Kyrie Irving
James Johnson (basketball, born 1987)
Patty Mills
Paul Millsap
Day%27Ron Sharpe
Cameron Thomas
New_York_Knicks
RJ Barrett
Alec Burks
Evan Fournier
Taj Gibson
Quentin Grimes
Kevin Knox 
Miles McBride
Nerlens Noel
Immanuel Quickley
Julius Randle
Mitchell Robinson
Derrick Rose
Luka Šamanić
Wayne Selden Jr.
Jericho Sims
Obi Toppin
Kemba Walker
Philadelphia_76ers
Charles Bassey
Seth Curry
Andre Drummond
Joel Embiid
Danny Green 
Tobias Harris
Aaron Henry 
Isaiah Joe
Furkan Korkmaz
Tyrese Maxey
Shake Milton
Georges Niang
Paul Reed 
Grant Riller
Ben 

KeyboardInterrupt: 

# Everything between here and Pandas is for doublechecking work above

In [11]:
html = requests.get('https://en.wikipedia.org/wiki/Los_Angeles_Lakers#', headers = {'User-Agent': ua}).text

In [12]:
print(len(playerList))

332


In [13]:
#find chunk of html in "Current Roster" section of article
rex = '<h3><span class="mw-headline" id="Current_roster">' + '.*?' + '</tbody></table>'
chunk1 = re.search(rex, html, re.DOTALL).group(0)
# chunk1

In [14]:
#Data cleasning step that became apparent after seeing data frame
chunk1 = re.sub('\(basketball\)','',chunk1)
chunk1 = re.sub('&amp;', '&', chunk1)

In [15]:
#find html chunk pertaining to each player
rex = '<td style="text-align:center' + '(.+?)' + '</td></tr>'
playerChunks = re.findall(rex, chunk1, re.DOTALL)
playerChunks[15]

';"><span data-sort-value="1.5&#160;!"><a href="/wiki/Guard_" class="mw-redirect" title="Guard ">G</a></span>\n</td>\n<td style="text-align:center;"><span data-sort-value="7000400000000000000♠">4</span>\n</td>\n<td style="text-align:left;"><a href="/wiki/Rajon_Rondo" title="Rajon Rondo">Rondo, Rajon</a>\n</td>\n<td style="text-align:right; white-space:nowrap;"><span data-sort-value="7000185420000000000♠"></span>6&#160;ft 1&#160;in (1.85&#160;m)\n</td>\n<td style="text-align:right; white-space:nowrap;">180&#160;lb (82&#160;kg)\n</td>\n<td style="text-align:center;">1986-02-22\n</td>\n<td style="text-align:left;"><a href="/wiki/University_of_Kentucky" title="University of Kentucky">Kentucky</a>\n'

In [18]:
#Create list of lists that contains stats for each player
playerList = []
for i in playerChunks:
    players = re.findall(r'title="(.+?)">', i) #finds [postion, name, alma mater] for each player
    heights = re.search(r'(\d\.\d\d)', i)       #finds weight of each player
    weights = re.search(r'">(\d\d\d)&#', i)     #finds heights
    heights = float(heights.group(1))
    weights = int(weights.group(1))
    players.append(heights)
    players.append(weights)
    for player in players[:1][:]:
        name = players[1]
        name = name.split()
        link = '<a href="(/wiki/' + re.escape(name[0]) + r'.+?)"'
        link = re.search(link, i)
        url = 'https://en.wikipedia.org' + link.group(1)
        if url[-1] == '_':
            url = url + '(basketball)'
        html = requests.get(url, headers = {'User-Agent': ua}).text
        nums = re.search(r'colspan="2"\S+?Career.+?</td>(.+?)</tr>', html, flags = re.DOTALL)
        ppg = re.search(r'<td>([\d\.]+)\n</td>', nums.group(1), flags = re.DOTALL)
        ppg = float(ppg.group(1))
        stats = re.findall(r'<td.+?([\d\.\,]+?)<', nums.group(1))
        stats = [x.replace(',','') for x in stats]
        statistics = [float(x) for x in stats]
        players.extend(statistics)
        players.append(ppg)
    playerList.append(players) 
playerList

[['Forward ',
  'Carmelo Anthony',
  'Syracuse University',
  2.01,
  238,
  1191.0,
  1117.0,
  35.0,
  0.447,
  0.353,
  0.814,
  6.3,
  2.8,
  1.0,
  0.5,
  23.0],
 ['Forward ',
  'Trevor Ariza',
  'University of California, Los Angeles',
  2.03,
  215,
  1094.0,
  758.0,
  29.7,
  0.423,
  0.352,
  0.732,
  4.8,
  2.1,
  1.5,
  0.3,
  10.5],
 ['Swingman',
  'Kent Bazemore',
  'Old Dominion University',
  1.93,
  195,
  618.0,
  296.0,
  21.0,
  0.416,
  0.356,
  0.724,
  3.2,
  1.8,
  1.0,
  0.5,
  8.5],
 ['Guard ',
  'Avery Bradley',
  'University of Texas at Austin',
  1.91,
  180,
  598.0,
  499.0,
  28.0,
  0.435,
  0.363,
  0.78,
  2.9,
  1.8,
  1.0,
  0.2,
  11.5],
 ['Guard ',
  'Chaundee Brown',
  'University of Michigan',
  1.96,
  215,
  112.0,
  74.0,
  24.6,
  0.435,
  0.352,
  0.812,
  4.3,
  1.1,
  0.4,
  0.2,
  9.8],
 ['Forward-center',
  'Anthony Davis',
  'University of Kentucky',
  2.08,
  253,
  564.0,
  559.0,
  34.4,
  0.514,
  0.312,
  0.799,
  10.2,
  2.3,
  1

In [17]:
players[:1][:]

['Guard ']

#  Get East Conference

In [22]:
fid_East=urllib.request.urlopen('https://en.wikipedia.org/wiki/Eastern_Conference_(NBA)')
webpage_East=fid_East.read().decode('utf-8')
print(webpage_East)

<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Eastern Conference (NBA) - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"30da5828-b5e2-47f6-86a3-49168be994a9","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Eastern_Conference_(NBA)","wgTitle":"Eastern Conference (NBA)","wgCurRevisionId":1056825681,"wgRevisionId":1056825681,"wgArticleId":1305847,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages using the EasyTimeline extension","Use mdy dates from December 2020","Articles needing additional references from 

# Finding East Stats

In [23]:
East_raw=re.findall(r'<td style="text-align: left">(.+?)</tr>',webpage_East,re.DOTALL)
East_raw
East_REE = ' '.join([str(elem) for elem in East_raw])
East_cities=[]
for line in East_raw:
    n = re.search(r'>([\w]+.\w+.*)<', line)
    n.group(0)
    East_cities.append(n.group(1))


East_REE=East_REE.replace("\n","")
East_REE=East_REE.replace("&","")
East_REE=East_REE.replace("#","")
East_Num=re.findall(r'<td>(.+?)</td>',East_REE,re.DOTALL)
East_Num


['49',
 '23',
 '.681',
 '–',
 '72',
 '48',
 '24',
 '.667',
 '1.0',
 '72',
 '46',
 '26',
 '.639',
 '3.0',
 '72',
 '41',
 '31',
 '.569',
 '8.0',
 '72',
 '41',
 '31',
 '.569',
 '8.0',
 '72',
 '40',
 '32',
 '.556',
 '9.0',
 '72',
 '36',
 '36',
 '.500',
 '13.0',
 '72',
 '34',
 '38',
 '.472',
 '15.0',
 '72',
 '34',
 '38',
 '.472',
 '15.0',
 '72',
 '33',
 '39',
 '.458',
 '16.0',
 '72',
 '31',
 '41',
 '.431',
 '18.0',
 '72',
 '27',
 '45',
 '.375',
 '22.0',
 '72',
 '22',
 '50',
 '.306',
 '27.0',
 '72',
 '21',
 '51',
 '.292',
 '28.0',
 '72',
 '20',
 '52',
 '.278',
 '29.0',
 '72']

# East 2021 Scores + Team

In [24]:
tt=[]

for i in range(0,len(East_Num),5):
    tt.append(East_Num[i:i+5])

#print(tt)

East_Score2021 = [list(a) for a in zip(East_cities, tt)]
East_Score2021

[['Philadelphia 76ers', ['49', '23', '.681', '–', '72']],
 ['Brooklyn Nets', ['48', '24', '.667', '1.0', '72']],
 ['Milwaukee Bucks', ['46', '26', '.639', '3.0', '72']],
 ['New York Knicks', ['41', '31', '.569', '8.0', '72']],
 ['Atlanta Hawks', ['41', '31', '.569', '8.0', '72']],
 ['Miami Heat', ['40', '32', '.556', '9.0', '72']],
 ['Boston Celtics', ['36', '36', '.500', '13.0', '72']],
 ['Washington Wizards', ['34', '38', '.472', '15.0', '72']],
 ['Indiana Pacers', ['34', '38', '.472', '15.0', '72']],
 ['Charlotte Hornets', ['33', '39', '.458', '16.0', '72']],
 ['Chicago Bulls', ['31', '41', '.431', '18.0', '72']],
 ['Toronto Raptors', ['27', '45', '.375', '22.0', '72']],
 ['Cleveland Cavaliers', ['22', '50', '.306', '27.0', '72']],
 ['Orlando Magic', ['21', '51', '.292', '28.0', '72']],
 ['Detroit Pistons', ['20', '52', '.278', '29.0', '72']]]

# Get West Conference

In [25]:
fid_West=urllib.request.urlopen('https://en.wikipedia.org/wiki/Western_Conference_(NBA)')
webpage_West=fid_West.read().decode('utf-8')
print(webpage_West)


<!DOCTYPE html>
<html class="client-nojs" lang="en" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Western Conference (NBA) - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ecb936d9-f477-4dfd-b086-b336b08f49a9","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Western_Conference_(NBA)","wgTitle":"Western Conference (NBA)","wgCurRevisionId":1056275464,"wgRevisionId":1056275464,"wgArticleId":28141073,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages using the EasyTimeline extension","Use mdy dates from December 2020","Articles needing additional references from

# Finding West Stats

In [26]:
West_raw=re.findall(r'<td style="text-align: left">(.+?)</tr>',webpage_West,re.DOTALL)
West_raw
West_REE = ' '.join([str(elem) for elem in West_raw])
West_cities=[]
for line in West_raw:
    n = re.search(r'>([\w]+.\w+.*)<', line)
    n.group(0)
    West_cities.append(n.group(1))


West_REE=West_REE.replace("\n","")
West_REE=West_REE.replace("&","")
West_REE=West_REE.replace("#","")
West_Num=re.findall(r'<td>(.+?)</td>',West_REE,re.DOTALL)
West_Num


['52',
 '20',
 '.722',
 '–',
 '72',
 '51',
 '21',
 '.708',
 '1.0',
 '72',
 '47',
 '25',
 '.653',
 '5.0',
 '72',
 '47',
 '25',
 '.653',
 '5.0',
 '72',
 '42',
 '30',
 '.583',
 '10.0',
 '72',
 '42',
 '30',
 '.583',
 '10.0',
 '72',
 '42',
 '30',
 '.583',
 '10.0',
 '72',
 '38',
 '34',
 '.528',
 '14.0',
 '72',
 '39',
 '33',
 '.542',
 '13.0',
 '72',
 '33',
 '39',
 '.458',
 '19.0',
 '72',
 '31',
 '41',
 '.431',
 '21.0',
 '72',
 '31',
 '41',
 '.431',
 '21.0',
 '72',
 '23',
 '49',
 '.319',
 '29.0',
 '72',
 '22',
 '50',
 '.306',
 '30.0',
 '72',
 '17',
 '55',
 '.236',
 '35.0',
 '72']

# West 2021 Scores + Team

In [27]:
pp=[]

for i in range(0,len(West_Num),5):
    pp.append(West_Num[i:i+5])

#print(tt)
West_Score2021 = [list(a) for a in zip(West_cities, pp)]
West_Score2021

[['Utah Jazz', ['52', '20', '.722', '–', '72']],
 ['Phoenix Suns', ['51', '21', '.708', '1.0', '72']],
 ['Denver Nuggets', ['47', '25', '.653', '5.0', '72']],
 ['Los Angeles Clippers', ['47', '25', '.653', '5.0', '72']],
 ['Dallas Mavericks', ['42', '30', '.583', '10.0', '72']],
 ['Portland Trail Blazers', ['42', '30', '.583', '10.0', '72']],
 ['Los Angeles Lakers', ['42', '30', '.583', '10.0', '72']],
 ['Memphis Grizzlies', ['38', '34', '.528', '14.0', '72']],
 ['Golden State Warriors', ['39', '33', '.542', '13.0', '72']],
 ['San Antonio Spurs', ['33', '39', '.458', '19.0', '72']],
 ['New Orleans Pelicans', ['31', '41', '.431', '21.0', '72']],
 ['Sacramento Kings', ['31', '41', '.431', '21.0', '72']],
 ['Minnesota Timberwolves', ['23', '49', '.319', '29.0', '72']],
 ['Oklahoma City Thunder', ['22', '50', '.306', '30.0', '72']],
 ['Houston Rockets', ['17', '55', '.236', '35.0', '72']]]

# Pandas Data Frame

In [19]:
import pandas as pd

In [20]:
df = pd.DataFrame(playerList, columns=['Position', 'Player', 'Alma Mater', 'Team', 'Height (m)', 'Weight (lb)', 'GP', 'GS', 'MPG', 'FG%', '3P%', 'FT%', 'RPG', 'APG', 'SPG', 'BPG', 'PPG'])
df

ValueError: 17 columns passed, passed data had 16 columns